In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import BalancedSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask = load_data(args)

load gdsc2


In [5]:
adj_coo = sp.coo_matrix(res)
pos_edges = np.vstack((adj_coo.row, adj_coo.col)).T

# ネガティブエッジ候補作成
adj_dense = adj_coo.toarray()

# ポジティブ（接続ありかつ null_mask==0）
pos_mask = np.logical_and(adj_dense == 1, null_mask == 0)
pos_row, pos_col = np.where(pos_mask)
pos_edges = np.vstack((pos_row, pos_col)).T
pos_labels = np.ones(len(pos_edges), dtype=int)

# ネガティブ（接続なしかつ null_mask==0）
neg_mask = np.logical_and(adj_dense == 0, null_mask == 0)
neg_row, neg_col = np.where(neg_mask)
neg_edges = np.vstack((neg_row, neg_col)).T
neg_labels = np.zeros(len(neg_edges), dtype=int)

# 統合
all_edges = np.vstack((pos_edges, neg_edges))
all_labels = np.concatenate((pos_labels, neg_labels))

In [6]:
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for i, (train_idx, test_idx) in enumerate(kf.split(all_edges)):
    train_edges = all_edges[train_idx]
    train_labels = all_labels[train_idx]
    test_edges = all_edges[test_idx]
    test_labels = all_labels[test_idx]

    sampler = BalancedSampler(
        edge_train=train_edges,
        label_train=train_labels,
        edge_test=test_edges,
        label_test=test_labels,
        adj_shape=res.shape,
        seed=i
    )
    
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.703975 auc:0.4890
epoch:  20 loss:0.219061 auc:0.9265
epoch:  40 loss:0.122278 auc:0.9392
epoch:  60 loss:0.084299 auc:0.9374
epoch:  80 loss:0.068109 auc:0.9359
epoch: 100 loss:0.059622 auc:0.9346
epoch: 120 loss:0.054271 auc:0.9333
epoch: 140 loss:0.050483 auc:0.9320
epoch: 160 loss:0.047657 auc:0.9307
epoch: 180 loss:0.045481 auc:0.9298
epoch: 200 loss:0.043798 auc:0.9291
epoch: 220 loss:0.042432 auc:0.9284
epoch: 240 loss:0.041304 auc:0.9278
epoch: 260 loss:0.042377 auc:0.9251
epoch: 280 loss:0.039855 auc:0.9264
epoch: 300 loss:0.039012 auc:0.9265
epoch: 320 loss:0.038417 auc:0.9258
epoch: 340 loss:0.037951 auc:0.9255
epoch: 360 loss:0.037590 auc:0.9252
epoch: 380 loss:0.037303 auc:0.9252
epoch: 400 loss:0.042838 auc:0.9244
epoch: 420 loss:0.037520 auc:0.9254
epoch: 440 loss:0.036849 auc:0.9249
epoch: 460 loss:0.036617 auc:0.9249
epoch: 480 loss:0.036472 auc:0.9248
epoch: 500 loss:0.036356 auc:0.9248
epoch: 520 loss:0.038665 auc:0.9246
epoch: 540 loss:0.036636 auc

In [8]:
true_datas.to_csv(f"true_{args.data}.csv")
predict_datas.to_csv(f"pred_{args.data}.csv")